# Download DAHITI
***

***Author:** Chus Casado Rodríguez*<br>
***Date:** 17-02-2025*<br>

**Description**<br>

This code downloads the target points available in the [DAHITI](https://dahiti.dgfi.tum.de/en/) dataset using its API version 2. 

It requires two inputs:
1. A TXT file (_api_key.txt_) with the API key associated to your DAHITI user.
2. A YAML file with the configuration: URL, type of point of interest (river, reservoir) and output path.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import requests
import json
import yaml
import geopandas as gpd
from shapely import Point
import pprint
from pathlib import Path

## Configuration

In [6]:
# configuration
with open('config.yml', 'r') as file:
    cfg = yaml.safe_load(file)
API_URL = cfg.get('api_url', 'https://dahiti.dgfi.tum.de/api/v2/')
TYPE = cfg.get('type', None)
COUNTRY = cfg.get('country', None)
PATH_DAHITI = Path(cfg.get('output_path', './'))

# personal API key
with open('api_key.txt', 'r') as txt:
    api_key = txt.readline().strip()

In [3]:
MAP_VAR_NAMES = {
    'target_name': 'name',
    'water_level_altimetry': 'level',
    'surface_area': 'area',
    'water_occurrence_mask': 'occurrence',
    'land_water_mask': 'land_water',
    'volume_variation': 'volume',
    'water_level_hypsometry': 'level_hyps'
}

## Targets

### Download

In [8]:
# requests targets
response = requests.post(
    url=f'{API_URL}/list-targets/',
    json={'api_key': api_key}
)

if response.status_code == 200:
    targets = json.loads(response.text)
else:
    error = json.loads(response.text)
    print('Error {0}:\t{1}'.format(error['code'], error['message']))

print('DAHITI contains {0} targets.'.format(len(targets['data'])))

DAHITI contains 11284 targets.


In [10]:
np.unique([target['type'].lower() for target in targets['data']])

array(['lake', 'reservoir', 'river', 'wetland'], dtype='<U9')

In [7]:
# filter targets by type
targets = [target for target in targets['data'] if target['type'].lower() == TYPE]
print('DAHITI contains {0} targets of type {1}.'.format(len(targets), TYPE))

### Reformat

In [7]:
# convert to DataFrame
target_list = []
variables = []
for reservoir in targets:
    serie = pd.concat((
        pd.Series({key: value for key, value in reservoir.items() if key != 'data_access'}),
        pd.Series(reservoir['data_access'])
    ), axis=0)
    variables += list(reservoir['data_access'])
    target_list.append(serie)
targets = pd.concat(target_list, axis=1).T
targets.dahiti_id = targets.dahiti_id.astype(int)
targets[['longitude', 'latitude']] = targets[['longitude', 'latitude']].astype(float)
variables = list(set(variables))
targets[variables] = targets[variables].astype(bool)
targets.set_index('dahiti_id', drop=True, inplace=True)

# convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(targets.longitude, targets.latitude)]
targets = gpd.GeoDataFrame(targets, geometry=geometry)
targets = targets.set_crs(epsg=4326)

# shorten variable names
targets.rename(columns=MAP_VAR_NAMES, inplace=True)

### Export

In [7]:
# export shapefile of targets
PATH_OUT = PATH_DAHITI / TYPE / 'targets'
PATH_OUT.mkdir(parents=True, exist_ok=True)
output_file = PATH_OUT / f'DAHITI_{TYPE}.shp'
targets.to_file(output_file, driver='ESRI Shapefile')
print(f'Shapefile of targets saved in:\t{output_file}')

DAHITI contains 11284 targets.
DAHITI contains 530 targets of type reservoir.
Shapefile of targets saved in:	Z:\nahaUsers\casadje\datasets\DAHITI\reservoir\targets\DAHITI_reservoir.shp
